In [1]:
import pandas as pd
import pickle
from rank_bm25 import BM25Okapi
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sentence_transformers import SentenceTransformer, util
import torch

/Library/Python/3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Users/borankahraman/Library/Python/3.9/lib/python/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [2]:
# Load the abstracts from the pickle file
with open('article_abstracts_quoted.pkl', 'rb') as f:
    abstracts = pickle.load(f)

# Prepare the BM25 model
tokenized_corpus = [abstract.split() for abstract in abstracts]
bm25 = BM25Okapi(tokenized_corpus)
doc2vec

scincl = SentenceTransformer('/Users/borankahraman/ITU/lcr/lcr_yeni/fine-tuned-scincl')

No sentence-transformers model found with name /Users/borankahraman/ITU/lcr/lcr_yeni/fine-tuned-scincl. Creating a new one with MEAN pooling.


In [3]:

# Query and rerank function
def query_and_rerank(query, bm25, scincl_model, top_k=10):
    # Tokenize the query
    tokenized_query = query.split()
    
    # Get BM25 top_k results
    bm25_scores = bm25.get_scores(tokenized_query)
    top_indices = bm25_scores.argsort()[-top_k:][::-1]
    top_abstracts = [abstracts[i] for i in top_indices]
    
    # Encode the query and abstracts
    query_embedding = scincl_model.encode(query, convert_to_tensor=True)
    abstract_embeddings = scincl_model.encode(top_abstracts, convert_to_tensor=True)
    
    # Compute cosine similarities
    cosine_scores = util.pytorch_cos_sim(query_embedding, abstract_embeddings)
    
    # Get the most similar abstract
    top_result_idx = torch.argmax(cosine_scores)
    
    return top_abstracts[top_result_idx]

# Example query
query = "The political system of a country defines the set of formal legal institutions that constitute a government or a state and establishes the distribution of power and resources among its citizens and government officials."

# Get the most suitable abstract for the query
most_suitable_abstract = query_and_rerank(query, bm25, scincl)
print(most_suitable_abstract)


Federal State, being part of governmental communities, houses two forms of states. These are "federal state" and "federated states". Federal state is also called "federation". Federation as a governmental system is accepted and implemented by many states all over the world. And federalism is a political system which emerged as a result of the organisation of federal state, and this system have a series of pecularities. In particular, when a mention is made of federalism, this calls to mind a sharing-out of govermental powers between federal state and federated states. And, on the other hand, in decentralization, some of the administrative and political powers are shared with the authorities which are not part of the central administrative organization. Decentralization involves two aspects, a political one and an administrative one. Both in federalism and decentralization, the governmental powers are shared between the central government and local govermental units. However, as there 